
>


## Import Libraries

In [1]:
# import libraries

#!pip install kaggle

import numpy as np
import pandas as pd
import zipfile
import kaggle

## Download and read CSV file

In [2]:
# download the dataset

#!kaggle datasets download -d nelgiriyewithana/countries-of-the-world-2023

In [3]:
# # extract the file from the downloaded zip file
# 
# zipfile_name = 'countries-of-the-world-2023.zip'
# with zipfile.ZipFile(zipfile_name, 'r') as file:
#     file.extractall()

In [4]:
# read in the csv file as a pandas dataframe
countries_load = pd.read_csv('world-data-2023.csv')

## Data Exploration

In [5]:
# view dataframe head with all columns
with pd.option_context('display.max_columns', None):
    print(countries_load.head())

       Country Density\n(P/Km2) Abbreviation Agricultural Land( %)  \
0  Afghanistan               60           AF                58.10%   
1      Albania              105           AL                43.10%   
2      Algeria               18           DZ                17.40%   
3      Andorra              164           AD                40.00%   
4       Angola               26           AO                47.50%   

  Land Area(Km2) Armed Forces size  Birth Rate  Calling Code  \
0        652,230           323,000       32.49          93.0   
1         28,748             9,000       11.78         355.0   
2      2,381,741           317,000       24.28         213.0   
3            468               NaN        7.20         376.0   
4      1,246,700           117,000       40.73         244.0   

  Capital/Major City Co2-Emissions     CPI CPI Change (%) Currency-Code  \
0              Kabul         8,672   149.9          2.30%           AFN   
1             Tirana         4,536  119.05  

In [6]:
# inspect dataframe's datatypes and missing values
countries_load.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 35 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Country                                    195 non-null    object 
 1   Density
(P/Km2)                            195 non-null    object 
 2   Abbreviation                               188 non-null    object 
 3   Agricultural Land( %)                      188 non-null    object 
 4   Land Area(Km2)                             194 non-null    object 
 5   Armed Forces size                          171 non-null    object 
 6   Birth Rate                                 189 non-null    float64
 7   Calling Code                               194 non-null    float64
 8   Capital/Major City                         192 non-null    object 
 9   Co2-Emissions                              188 non-null    object 
 10  CPI                       

## Raw Dataframe Summary
- 195 rows, 35 columns
- Column names should be standardized.
- Some object (string) type columns should be numeric.
- Most columns have missing values.

## Cleaning the Dataframe

In [7]:
# copy the dataframe to begin cleaning
countries = countries_load.copy()

In [8]:
# standardize column names
countries.columns = (
    countries.columns
    .str.strip()  # remove leading/trailing whitespace
    .str.replace(r'\s+', '_', regex=True)  # replace spaces/newlines with underscores
    .str.replace(r'[^\w_]', '', regex=True)  # remove special characters like (, %, :)
    .str.lower()  # lowercase for consistency
    .str.rstrip('_')  # strip trailing underscores
)

# inspect progress of column rename
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 35 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   country                               195 non-null    object 
 1   density_pkm2                          195 non-null    object 
 2   abbreviation                          188 non-null    object 
 3   agricultural_land                     188 non-null    object 
 4   land_areakm2                          194 non-null    object 
 5   armed_forces_size                     171 non-null    object 
 6   birth_rate                            189 non-null    float64
 7   calling_code                          194 non-null    float64
 8   capitalmajor_city                     192 non-null    object 
 9   co2emissions                          188 non-null    object 
 10  cpi                                   178 non-null    object 
 11  cpi_change         

In [9]:
# these column names need further refinement for standardization
column_rename = {'agricultural_land': 'agricultural_land_pct',
                'land_areakm2': 'land_area_km2',
                'capitalmajor_city': 'capital_major_city',
                'co2emissions': 'co2_emissions',
                'cpi_change': 'cpi_change_pct',
                'currencycode': 'currency_code',
                'forested_area': 'forested_area_pct',
                'gross_primary_education_enrollment': 'gross_primary_edu_pct',
                'gross_tertiary_education_enrollment': 'gross_tertiary_edu_pct',
                'out_of_pocket_health_expenditure': 'oop_health_expend',
                'population_labor_force_participation': 'pop_labor_force_participation_pct',
                'tax_revenue': 'tax_revenue_pct'
                }

# rename the columns
countries.rename(columns=column_rename, inplace=True)

In [10]:
# identify the columns that should be converted from object (string) datatype to numeric
string_cols = ['density_pkm2', 'agricultural_land_pct',
       'land_area_km2', 'armed_forces_size', 
       'co2_emissions', 'cpi', 'cpi_change_pct', 
       'forested_area_pct', 'gasoline_price', 'gdp', 'gross_primary_edu_pct',
       'gross_tertiary_edu_pct', 'minimum_wage',
       'oop_health_expend', 'population',
       'pop_labor_force_participation_pct', 'tax_revenue_pct',
       'total_tax_rate', 'unemployment_rate', 'urban_population']

# clean string data (remove special characters) and change to numeric
for col in string_cols:
    countries[col] = (countries[col]
                                .str.replace('%', '', regex=False)
                                .str.replace('nan', 'NaN', regex=False)
                                .str.replace(',', '', regex=False)
                                .str.replace('$', '', regex=False)
                                .str.strip()
                     )
    countries[col] = pd.to_numeric(countries[col], errors='coerce')  # convert to numeric
    

In [11]:
# verify that the correct columns were changed to numeric
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 35 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   country                            195 non-null    object 
 1   density_pkm2                       195 non-null    int64  
 2   abbreviation                       188 non-null    object 
 3   agricultural_land_pct              188 non-null    float64
 4   land_area_km2                      194 non-null    float64
 5   armed_forces_size                  171 non-null    float64
 6   birth_rate                         189 non-null    float64
 7   calling_code                       194 non-null    float64
 8   capital_major_city                 192 non-null    object 
 9   co2_emissions                      188 non-null    float64
 10  cpi                                178 non-null    float64
 11  cpi_change_pct                     179 non-null    float64

In [12]:
# Some of the numeric columns should be integer datatype.
# list of columns that should be integer datatype
columns_to_integer = ['land_area_km2', 'armed_forces_size', 'calling_code', 'co2_emissions',
                     'maternal_mortality_ratio', 'population', 'urban_population']

# change the datatype of these columns to integer
for col in columns_to_integer:
    countries[col] = countries[col].astype('Int64')

In [13]:
# print dataset head with all columns
with pd.option_context('display.max_columns', None):
    print(countries.head())

       country  density_pkm2 abbreviation  agricultural_land_pct  \
0  Afghanistan            60           AF                   58.1   
1      Albania           105           AL                   43.1   
2      Algeria            18           DZ                   17.4   
3      Andorra           164           AD                   40.0   
4       Angola            26           AO                   47.5   

   land_area_km2  armed_forces_size  birth_rate  calling_code  \
0         652230             323000       32.49            93   
1          28748               9000       11.78           355   
2        2381741             317000       24.28           213   
3            468               <NA>        7.20           376   
4        1246700             117000       40.73           244   

  capital_major_city  co2_emissions     cpi  cpi_change_pct currency_code  \
0              Kabul           8672  149.90             2.3           AFN   
1             Tirana           4536  119.05   

In [14]:
# An encoding issue was mentioned in the discussion board for this dataset.

# search and print all rows with corrupted data (�) 
mask = countries.map(lambda x: '�' in str(x))
rows_with_bad_chars = countries[mask.any(axis=1)]

with pd.option_context('display.max_columns', None):
    print(rows_with_bad_chars)

          country  density_pkm2 abbreviation  agricultural_land_pct  \
23         Brazil            25           BR                   33.9   
31       Cameroon            56           CM                   20.6   
37       Colombia            46           CO                   40.3   
40     Costa Rica           100           CR                   34.5   
43         Cyprus           131           CY                   12.2   
76        Iceland             3           IS                   18.7   
104      Maldives          1802           MV                   26.3   
112       Moldova           123           MD                   74.2   
136      Paraguay            18           PY                   55.1   
150  S�����������           228           ST                   50.7   
168        Sweden            25           SE                    7.4   
169   Switzerland           219           CH                   38.4   
175          Togo           152           TG                   70.2   
176   

In [26]:
# dictionary of corrections for capital cities
corrections_capital_cities = {
    23:'Brasília',
    31:'Yaoundé',
    37:'Bogotá',
    40:'San José',
    43:'Nicosia',
    76:'Reykjavík',
    104:'Malé',
    112:'Chișinău',
    136:'Asunción',
    150:'São Tomé',
    168:'Stockholm',
    169:'Bern',
    175:'Lomé',
    176:'Nukuʻalofa'
}

# apply corrections to capital cities
for index, value in corrections_capital_cities.items():
    countries.at[index, 'capital_major_city'] = value

# dictionary of corrections for largest cities
corrections_largest_cities = {
    23:'São Paulo',
    37:'Bogotá',
    40:'San José',
    43:'Statos',
    76:'Reykjavík',
    104:'Malé',
    112:'Chișinău',
    150:'São Tomé',
    168:'Stockholm',
    169:'Zurich',
    175:'Lomé',
    176:'Nukuʻalofa'
}
# apply corrections to largest cities
for index, value in corrections_largest_cities.items():
    countries.at[index, 'largest_city'] = value

# correction for country name (São Tomé and Príncipe deduced based on calling code)
countries.loc[150, 'country'] = 'São Tomé and Príncipe'

# verify all corrupted data is fixed 
mask = countries.map(lambda x: '�' in str(x))
rows_with_bad_chars = countries[mask.any(axis=1)]
with pd.option_context('display.max_columns', None):
    print(rows_with_bad_chars)

Empty DataFrame
Columns: [country, density_pkm2, abbreviation, agricultural_land_pct, land_area_km2, armed_forces_size, birth_rate, calling_code, capital_major_city, co2_emissions, cpi, cpi_change_pct, currency_code, fertility_rate, forested_area_pct, gasoline_price, gdp, gross_primary_edu_pct, gross_tertiary_edu_pct, infant_mortality, largest_city, life_expectancy, maternal_mortality_ratio, minimum_wage, official_language, oop_health_expend, physicians_per_thousand, population, pop_labor_force_participation_pct, tax_revenue_pct, total_tax_rate, unemployment_rate, urban_population, latitude, longitude]
Index: []


In [16]:
# handling missing values
countries.isnull().sum()

country                               0
density_pkm2                          0
abbreviation                          7
agricultural_land_pct                 7
land_area_km2                         1
armed_forces_size                    24
birth_rate                            6
calling_code                          1
capital_major_city                    3
co2_emissions                         7
cpi                                  17
cpi_change_pct                       16
currency_code                        15
fertility_rate                        7
forested_area_pct                     7
gasoline_price                       20
gdp                                   2
gross_primary_edu_pct                 7
gross_tertiary_edu_pct               12
infant_mortality                      6
largest_city                          6
life_expectancy                       8
maternal_mortality_ratio             14
minimum_wage                         45
official_language                     5


## Handle Missing Values

In [ ]:
# TO DO: abbreviations should be two letters not three

In [17]:
# find missing values in the abbreviation column
countries[countries['abbreviation'].isnull()]['country']

39              Republic of the Congo
56                           Eswatini
73                       Vatican City
81                Republic of Ireland
119                           Namibia
128                   North Macedonia
133    Palestinian National Authority
Name: country, dtype: object

In [18]:
# dictionary of country names and abbreviations to fill missing values
updated_abbreviations = {'Republic of the Congo': 'RCG',
                        'Eswatini': 'ESW',
                        'Vatican City': 'VAC',
                        'Republic of Ireland': 'ROI',
                        'Namibia': 'NAM',
                        'North Macedonia': 'NOM',
                        'Palestinian National Authority': 'PNA'
                        }

# Update only missing abbreviations
countries['abbreviation'] = countries.apply(
    lambda row: updated_abbreviations.get(row['country'], row['abbreviation'])
    if pd.isna(row['abbreviation']) else row['abbreviation'],
    axis=1
)

# verify that the missing values were filled in correctly
print(countries[countries['abbreviation'].isin(['RCG', 'ESW', 'VAC', 'ROI', 'NAM', 'NOM', 'PNA'])]
      [['country', 'abbreviation']])

                            country abbreviation
39            Republic of the Congo          RCG
56                         Eswatini          ESW
73                     Vatican City          VAC
81              Republic of Ireland          ROI
119                         Namibia          NAM
128                 North Macedonia          NOM
133  Palestinian National Authority          PNA


In [19]:
# verify that there are no duplicates in the abbreviation column
print('Are there any duplicated values?  ', countries['abbreviation'].duplicated().any())

# verify that there are no missing values in the abbreviation column
countries.isnull().sum()

Are there any duplicated values?   False


country                               0
density_pkm2                          0
abbreviation                          0
agricultural_land_pct                 7
land_area_km2                         1
armed_forces_size                    24
birth_rate                            6
calling_code                          1
capital_major_city                    3
co2_emissions                         7
cpi                                  17
cpi_change_pct                       16
currency_code                        15
fertility_rate                        7
forested_area_pct                     7
gasoline_price                       20
gdp                                   2
gross_primary_edu_pct                 7
gross_tertiary_edu_pct               12
infant_mortality                      6
largest_city                          6
life_expectancy                       8
maternal_mortality_ratio             14
minimum_wage                         45
official_language                     5


In [20]:
# life_expectancy column has 8 missing values
# calculate the median of all existing values
median_life_expectancy = countries['life_expectancy'].median()

# fill missing values with median
countries['life_expectancy'] = countries['life_expectancy'].fillna(median_life_expectancy)

# verify that there are no missing values in the abbreviation column
countries['life_expectancy'].isnull().sum()

0

In [30]:
# print sample of dataset with all columns
with pd.option_context('display.max_columns', None):
    print(countries.sample(10))

                 country  density_pkm2 abbreviation  agricultural_land_pct  \
18                 Benin           108           BJ                   33.3   
0            Afghanistan            60           AF                   58.1   
178              Tunisia            76           TN                   64.8   
5    Antigua and Barbuda           223           AG                   20.5   
42                  Cuba           106           CU                   59.9   
16               Belgium           383           BE                   44.6   
4                 Angola            26           AO                   47.5   
171           Tajikistan            68           TJ                   34.1   
170                Syria            95           SY                   75.8   
90                Kuwait           240           KW                    8.4   

     land_area_km2  armed_forces_size  birth_rate  calling_code  \
18          112622              12000       36.22           229   
0      